In [1]:
import os
import hmac
import hashlib

In [39]:
class IoTDevice:
    def __init__(self, device_id, shared_vault):
        self.device_id = device_id
        self.vault = shared_vault.copy()

    def create_connection_request(self, session_id):
        print(f"IoTDevice: Creating connection request with session_id: {session_id} and device_id: {self.device_id}")
        return {"device_id": self.device_id, "session_id": session_id}

    def generate_response(self, challenge_indices, r1):
        print(f"IoTDevice: Generating response with challenge_indices: {challenge_indices} and r1: {r1}")
        # Step 1: Compute k1 as XOR of keys in the indices
        k1 = self.vault[challenge_indices[0]]
        for i in challenge_indices[1:]:
            k1 = bytes(a ^ b for a, b in zip(k1, self.vault[i]))
        print("IoTDevice: K1 (XOR Vault): ", k1)
        # Step 2: Generate t1 (random number for session key generation)
        t1 = os.urandom(len(r1))
        print("IoTDevice: R1 from server: ", r1)
        print("IoTDevice: T1: ", t1)

        # Step 3: Concatenate r1 and t1
        concat_r1_t1 = r1 + t1
        print("IoTDevice: Concatenated R1 and T1: ", concat_r1_t1)

        # Step 5: Generate C2
        c2 = self.generate_challenge()
        print("IoTDevice: Generated C2: ", c2)
        # Step 6: Concatenate response and c2
        final_response = hmac.new(k1,concat_r1_t1 + bytes(c2[0]) + bytes(c2[1]))
        print("IoTDevice: Final Response: ", final_response)
        return {"response": final_response}
    
    def generate_challenge(self, p=3):
        indices = list(set(os.urandom(1)[0] % len(self.vault) for _ in range(p)))
        while len(indices) < p:
            indices.append(os.urandom(1)[0] % len(self.vault))  # Ensure distinct indices
        r1 = os.urandom(8)  # Random number
        print(f"IoTDevice: Client selected indices indices: {indices}")
        print(f"IoTDevice: Random number r1: {r1}")
        return indices, r1

    def update_vault(self, hmac_key):
        self.vault = [
            hmac.new(hmac_key, key, hashlib.sha256).digest()
            for key in self.vault
        ]

In [36]:
class IoTServer:
    def __init__(self, shared_vault, valid_device_ids):
        self.vault = shared_vault.copy()
        self.valid_device_ids = valid_device_ids
        self.C1 = None
        self.C2 = None
        self.r1 = None
        self.r2 = None

    def handle_connection_request(self, request):
        device_id = request.get("device_id")
        session_id = request.get("session_id")
        if device_id in self.valid_device_ids:
            print(f"IoTServer: Connection request accepted for Device ID: {device_id}, Session ID: {session_id}")
            return True
        else:
            print(f"IoTServer: Connection request denied for Device ID: {device_id}")
            return False

    def generate_challenge(self, p=3):
        indices = list(set(os.urandom(1)[0] % len(self.vault) for _ in range(p)))
        while len(indices) < p:
            indices.append(os.urandom(1)[0] % len(self.vault))  # Ensure distinct indices
        r1 = os.urandom(8)  # Random number
        print(f"IoTServer: Server selected indices indices: {indices}")
        print(f"IoTServer: Random number r1: {r1}")
        return indices, r1

    def verify_response(self, challenge_indices, r1, device_response, t1):
        # Step 1: Compute k1 as XOR of keys in the indices
        k1 = self.vault[challenge_indices[0]]
        for i in challenge_indices[1:]:
            k1 = bytes(a ^ b for a, b in zip(k1, self.vault[i]))

        # Step 2: Concatenate r1 and t1
        concat_r1_t1 = r1 + t1

        # Step 3: Generate expected response
        expected_response = hmac.new(k1, concat_r1_t1, hashlib.sha256).digest()

        return expected_response == device_response

    def update_vault(self, hmac_key):
        self.vault = [
            hmac.new(hmac_key, key, hashlib.sha256).digest()
            for key in self.vault
        ]

In [37]:
# Shared initialization
VAULT_SIZE = 10
KEY_SIZE = 16
shared_vault = [os.urandom(KEY_SIZE) for _ in range(VAULT_SIZE)]
print("Shared Vault: ")
for i, key in enumerate(shared_vault):
    print(f"Index {i}: {key}")
valid_device_ids = {"Device_1", "Device_2"}  # List of valid device IDs

# Create device and server with the shared vault
device = IoTDevice(device_id="Device_1", shared_vault=shared_vault)
server = IoTServer(shared_vault=shared_vault, valid_device_ids=valid_device_ids)

Shared Vault: 
Index 0: b'^\xc8&b\x05\xd9\x88Z\xc1\xc2\x18\xf7\xf6\xbd%\xf5'
Index 1: b'\x86\xec1t\xaa\xee\xddt\xe4\x14 |J\x94c\x98'
Index 2: b'$4\xa76\xe4+\x06\x82\xafD*\xdd\x9eH\xdcB'
Index 3: b'\xd1\xd6\n\xbeO\xd4\x95\xc5!\x12#\x8cob\xf1]'
Index 4: b'\xb2\xce;\xb8\x8c\xbc\xef"<i\xbb\xa0\xf3\xabb\x18'
Index 5: b'\xc1\xbb\xbc\xc9i\x81\xa3\xebCa\x18\x93\xd8q\x94\xbb'
Index 6: b'\xa6\xcf\x1c7\xf5\xdd\xd4\x8clo\xac\x06\xe7\x0e\xdb='
Index 7: b'Or\xd3\x9e\xe5dk`\x93g\xc2R\xb3+8\x03'
Index 8: b']\xa5\xf4+9^\xfc>\x86\xa7*\xe5\xadsJG'
Index 9: b'X9\xb5\xcd\x87\xb3\x06\xf3\xdc\x8dm\xf6\xee\xbb\xdc\xe8'


In [40]:
# Authentication process
print("Starting Authentication Process")

# Step 1: Device sends connection request
session_id = "session_123"
connection_request = device.create_connection_request(session_id)

# Step 2: Server verifies device ID and sends challenge if valid
if server.handle_connection_request(connection_request):
    challenge_indices, r1 = server.generate_challenge()

    # Step 3: Device generates response
    response_packet = device.generate_response(challenge_indices, r1)
    device_response = response_packet["response"]
    print("[AUTH] Device Response: ", device_response)
    t1 = response_packet["t1"]
    print("[AUTH] Device T1: ", t1)

    # Step 4: Server verifies response
    if server.verify_response(challenge_indices, r1, device_response, t1):
        print("Authentication Successful!")

        # Step 5: Vault update after successful authentication
        print("Updating vaults...")
        data_to_update = b"session_data"
        hmac_key = hmac.new(data_to_update, b"update_key", hashlib.sha256).digest()
        device.update_vault(hmac_key)
        server.update_vault(hmac_key)
        print("Vaults updated successfully!")
    else:
        print("Authentication Failed.")
else:
    print("IoTServer: Connection denied.")

Starting Authentication Process
IoTDevice: Creating connection request with session_id: session_123 and device_id: Device_1
IoTServer: Connection request accepted for Device ID: Device_1, Session ID: session_123
IoTServer: Server selected indices indices: [9, 3, 6]
IoTServer: Random number r1: b'\xd9\xf9\xeeU\xc4\xa9\xa4\x1d'
IoTDevice: Generating response with challenge_indices: [9, 3, 6] and r1: b'\xd9\xf9\xeeU\xc4\xa9\xa4\x1d'
IoTDevice: K1 (XOR Vault):  b'/ \xa3D=\xbaG\xba\x91\xf0\xe2|f\xd7\xf6\x88'
IoTDevice: R1 from server:  b'\xd9\xf9\xeeU\xc4\xa9\xa4\x1d'
IoTDevice: T1:  b'\xb4"\x8d\xddt\xack\xa9'
IoTDevice: Concatenated R1 and T1:  b'\xd9\xf9\xeeU\xc4\xa9\xa4\x1d\xb4"\x8d\xddt\xack\xa9'
IoTDevice: Challenge Response:  b'lt\x07\x14\xa6\x13*\xe7\xd6\xa5\xe35)\x8b\x90\x8a\x0b\xd0\xce\x19\x8ez+= T=\xa1\xb8\xd2\xb36'
IoTDevice: Client selected indices indices: [1, 5, 6]
IoTDevice: Random number r1: b'\xb5qri\x07\x8fO7'
IoTDevice: Generated C2:  ([1, 5, 6], b'\xb5qri\x07\x8fO7')
IoT

KeyError: 't1'